In [17]:
import tensorflow as tf

#reset tensorflow graph
tf.reset_default_graph()

#Import Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#Training & Test Data
trainX = mnist.train.images
trainY = mnist.train.labels

testX = mnist.test.images
testY = mnist.test.labels

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [18]:
#Classifier Parameters
logs_path='/tmp/session4/v2'
save_path=logs_path + '/'
learning_rate = 0.003 #0.003, 0.0001, 0.0002
n_features = 784
n_classes = 10
batch_size = 100
training_epochs = 100

In [19]:
K = 200
L = 100
M = 60
N = 30

In [20]:
#Build Graph - Define Tensorflow Operations

with tf.name_scope('input'):
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

In [21]:
#Initialize a layer 1 of 200 neurons
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.truncated_normal([n_features, K] ,stddev=0.1))
    b1 = tf.Variable(tf.ones([K]))
    Y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

In [22]:
#Initialize a layer 2 of 100 neurons
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.truncated_normal([K, L] ,stddev=0.1))
    b2 = tf.Variable(tf.ones([L]))
    Y2 = tf.nn.sigmoid(tf.matmul(Y1, W2) + b2)

In [23]:
#Initialize a layer 3 of 60 neurons
with tf.name_scope("Layer3"):
    W3 = tf.Variable(tf.truncated_normal([L, M] ,stddev=0.1))
    b3 = tf.Variable(tf.ones([M]))
    Y3 = tf.nn.sigmoid(tf.matmul(Y2, W3) + b3)

In [24]:
#Initialize a layer 4 of 30 neurons
with tf.name_scope("Layer4"):
    W4 = tf.Variable(tf.truncated_normal([M, N] ,stddev=0.1))
    b4 = tf.Variable(tf.ones([N]))
    Y4 = tf.nn.sigmoid(tf.matmul(Y3, W4) + b4)

In [25]:
# implement model
with tf.name_scope("Output"):
    # y is our prediction
    W = tf.Variable(tf.truncated_normal([N, n_classes] ,stddev=0.1))
    b = tf.Variable(tf.zeros([n_classes]))   
    #y = tf.nn.softmax(tf.matmul(Y4, W) + b)
    Ylogits = tf.matmul(Y4, W) + b
    y = tf.nn.softmax(Ylogits)

In [26]:
# specify cost function
with tf.name_scope('Loss'):
    # this is our cost
    #cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=y_)
    cross_entropy = tf.reduce_mean(cross_entropy)*100

In [27]:
# specify optimizer
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    learn_rate = tf.Variable(learning_rate)
    train_op = tf.train.GradientDescentOptimizer(learn_rate).minimize(cross_entropy)

In [28]:
with tf.name_scope('Accuracy'):
    # Prediction
    prediction = tf.argmax(y,1,name="Predict")
    #Accuracy
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")

In [29]:
# create a summary for our cost and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

In [30]:
#Create a Saver to save the graph
saver = tf.train.Saver()

In [31]:
#Start Graph execution
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(trainX.shape[0]/batch_size)
        
        for i in range(batch_count):
            batch_x  = trainX[i*batch_size:i*batch_size+batch_size]
            batch_y  = trainY[i*batch_size:i*batch_size+batch_size]

            # perform the operations we defined earlier on batch
            _,acc,loss = sess.run([train_op, training_accuracy,training_loss], feed_dict={x: batch_x, y_: batch_y})
            
            #log training accuracy and loss
            writer.add_summary(acc, epoch * batch_count + i)
            writer.add_summary(loss, epoch * batch_count + i)    
                        
        #Test loss and accuracy
        acc,loss,a_loss = sess.run([test_accuracy,test_loss,cross_entropy],
                                   feed_dict={x: testX, y_: testY})
        writer.add_summary(acc, epoch * batch_count + i)
        writer.add_summary(loss, epoch * batch_count + i)
        print ('Test loss at step: ', epoch, ' is ', a_loss) 
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
                
    print ("Accuracy: ", accuracy.eval(feed_dict={x: testX, y_: testY}))
    
    #Save the model
    saver.save(sess, save_path + "model.ckpt")
    print ("done")

Test loss at step:  0  is  230.172
Epoch:  0
Test loss at step:  1  is  230.096
Test loss at step:  2  is  229.929
Test loss at step:  3  is  222.904
Test loss at step:  4  is  160.62
Test loss at step:  5  is  117.283
Epoch:  5
Test loss at step:  6  is  80.6451
Test loss at step:  7  is  58.8372
Test loss at step:  8  is  47.7768
Test loss at step:  9  is  39.9217
Test loss at step:  10  is  33.2838
Epoch:  10
Test loss at step:  11  is  28.1963
Test loss at step:  12  is  24.8707
Test loss at step:  13  is  22.455
Test loss at step:  14  is  20.4755
Test loss at step:  15  is  18.849
Epoch:  15
Test loss at step:  16  is  17.5824
Test loss at step:  17  is  16.6293
Test loss at step:  18  is  15.8968
Test loss at step:  19  is  15.2794
Test loss at step:  20  is  14.7145
Epoch:  20
Test loss at step:  21  is  14.1965
Test loss at step:  22  is  13.7388
Test loss at step:  23  is  13.3216
Test loss at step:  24  is  12.9344
Test loss at step:  25  is  12.5913
Epoch:  25
Test loss at 